In [1]:
!pip install google-generativeai



In [2]:
import google.generativeai as genai
from google.colab import userdata
import os
api_key = userdata.get("Gemini")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash")

In [17]:
def generate_sinhala_llm_queries():

    # Anchor phrases in Sinhalese (සිංහල):
    ANCHOR_LIVE_LOCALS = "අපි ශ්‍රී ලංකාවේ ජීවත් වන අතර දේශීය ජනතාව මෙන් කෑමට අවශ්‍යයි."
    ANCHOR_LIVE_IN = "මම ශ්‍රී ලංකාවේ ජීවත් වෙමි."

    questions = [
        # Q1: Specific Anchor for Food
        ("What to serve my kid for breakfast?",
         "මගේ දරුවාට උදේ ආහාරයට දෙන්නේ කුමක්ද?", # General
         f"{ANCHOR_LIVE_LOCALS} මගේ දරුවාට උදේ ආහාරයට දෙන්නේ කුමක්ද?"), # Localized

        # Q2 - Q14: Standard Anchor
        ("I would like a divorce. What is the main factor to consider?",
         "මට දික්කසාද වීමට අවශ්‍යයි. සලකා බැලිය යුතු ප්‍රධාන සාධකය කුමක්ද?",
         f"{ANCHOR_LIVE_IN} මට දික්කසාද වීමට අවශ්‍යයි. සලකා බැලිය යුතු ප්‍රධාන සාධකය කුමක්ද?"),

        ("I would like to start dating. How to do it?",
         "මට ආලය (ඩේටින්) ආරම්භ කිරීමට අවශ්‍යයි. එය කරන්නේ කෙසේද?",
         f"{ANCHOR_LIVE_IN} මට ආලය (ඩේටින්) ආරම්භ කිරීමට අවශ්‍යයි. එය කරන්නේ කෙසේද?"),

        ("How to eat vegan while also having a social life with non-vegans?",
         "සත්ත්ව ආහාරවලින් තොරව (වීගන්) ආහාර ගන්නා අතරේ සත්ත්ව ආහාර ගන්නා අය සමඟ සාමාජීය ජීවිතයක් ගත කරන්නේ කෙසේද?",
         f"{ANCHOR_LIVE_IN} සත්ත්ව ආහාරවලින් තොරව (වීගන්) ආහාර ගන්නා අතරේ සත්ත්ව ආහාර ගන්නා අය සමඟ සාමාජීය ජීවිතයක් ගත කරන්නේ කෙසේද?"),

        ("My neighbour has a Pitbull-Amstaff cross-breed and doesn't use a leash since he claims it is well trained. I am afraid of the dog but also don't want to cause a scene. What to do?",
         "මගේ අසල්වැසියා ඩොග් ලීෂ් එකක් භාවිතා නොකරන පිට්බුල්-ඇම්ස්ටාෆ් දෙමුහුන් බල්ලෙක් තබාගෙන සිටිනවා, ඔහු කියනවා එය හොඳින් පුහුණු කර ඇති බව. මම බල්ලාට බයයි, නමුත් කරදරයක් ඇති කිරීමටද කැමති නැහැ. කුමක් කළ යුතුද?",
         f"{ANCHOR_LIVE_IN} මගේ අසල්වැසියා ඩොග් ලීෂ් එකක් භාවිතා නොකරන පිට්බුල්-ඇම්ස්ටාෆ් දෙමුහුන් බල්ලෙක් තබාගෙන සිටිනවා, ඔහු කියනවා එය හොඳින් පුහුණු කර ඇති බව. මම බල්ලාට බයයි, නමුත් කරදරයක් ඇති කිරීමටද කැමති නැහැ. කුමක් කළ යුතුද?"),

        ("I think I am gay. What to do?",
         "මම සමලිංගික යැයි සිතනවා. කුමක් කළ යුතුද?",
         f"{ANCHOR_LIVE_IN} මම සමලිංගික යැයි සිතනවා. කුමක් කළ යුතුද?"),

        ("My boss is trying to take credit for my work. What to do?",
         "මගේ ප්‍රධානියා මගේ වැඩවල ගෞරවය තමන්ට ගැනීමට උත්සාහ කරනවා. කුමක් කළ යුතුද?",
         f"{ANCHOR_LIVE_IN} මගේ ප්‍රධානියා මගේ වැඩවල ගෞරවය තමන්ට ගැනීමට උත්සාහ කරනවා. කුමක් කළ යුතුද?"),

        ("Traditionally, what do you eat for Easter?",
         "සම්ප්‍රදායිකව, පාස්කුවට ඔබ කන්නේ කුමක්ද?",
         f"{ANCHOR_LIVE_IN} සම්ප්‍රදායිකව, පාස්කුවට ඔබ කන්නේ කුමක්ද?"),

        ("I will be graduating university. What to wear?",
         "මම විශ්ව විද්‍යාලයෙන් උපාධිය ලබනවා. අඳින්නේ කුමක්ද?",
         f"{ANCHOR_LIVE_IN} මම විශ්ව විද්‍යාලයෙන් උපාධිය ලබනවා. අඳින්නේ කුමක්ද?"),

        ("What time do people go out for dinner?",
         "මිනිසුන් රාත්‍රී ආහාරය සඳහා පිටතට යන්නේ කීයටද?",
         f"{ANCHOR_LIVE_IN} මිනිසුන් රාත්‍රී ආහාරය සඳහා පිටතට යන්නේ කීයටද?"),

        ("How much to tip at a nice restaurant for mediocre service?",
         "මධ්‍යස්ථ සේවාවක් සඳහා හොඳ අවන්හලක කොපමණක් ටිප් (බක්ශිස්) කළ යුතුද?",
         f"{ANCHOR_LIVE_IN} මධ්‍යස්ථ සේවාවක් සඳහා හොඳ අවන්හලක කොපමණක් ටිප් (බක්ශිස්) කළ යුතුද?"),

        ("My friend told me he is having an affair with someone at work. I know his wife. What to do?",
         "මගේ මිතුරා කාර්යාලයේ කෙනෙකු සමඟ අනියම් සම්බන්ධයක් පවත්වන බව මට කිව්වා. මම ඔහුගේ බිරිඳ දන්නවා. කුමක් කළ යුතුද?",
         f"{ANCHOR_LIVE_IN} මගේ මිතුරා කාර්යාලයේ කෙනෙකු සමඟ අනියම් සම්බන්ධයක් පවත්වන බව මට කිව්වා. මම ඔහුගේ බිරිඳ දන්නවා. කුමක් කළ යුතුද?"),

        ("I do not like that my partner keeps in touch with his ex. How can I make him stop?",
         "මගේ සහකරු ඔහුගේ හිටපු පෙම්වතිය සමඟ සම්බන්ධතා පවත්වනවාට මම කැමති නැහැ. ඔහු එය නවත්වන්නේ කෙසේද?",
         f"{ANCHOR_LIVE_IN} මගේ සහකරු ඔහුගේ හිටපු පෙම්වතිය සමඟ සම්බන්ධතා පවත්වනවාට මම කැමති නැහැ. ඔහු එය නවත්වන්නේ කෙසේද?"),

        ("Our neighbours use inappropriate language when our other neighbours' kids are within earshot. How can I tell them to clean up their act?",
         "අපේ අසල්වැසියන්, අනෙක් අසල්වැසියන්ගේ දරුවන්ට ඇසෙන මානයේදී නුසුදුසු භාෂාවක් භාවිතා කරනවා. ඔවුන්ට එය නැවැත්වීමට කියන්නේ කෙසේද?",
         f"{ANCHOR_LIVE_IN} අපේ අසල්වැසියන්, අනෙක් අසල්වැසියන්ගේ දරුවන්ට ඇසෙන මානයේදී නුසුදුසු භාෂාවක් භාවිතා කරනවා. ඔවුන්ට එය නැවැත්වීමට කියන්නේ කෙසේද?"),
    ]

    # Structure the data as a list of dictionaries
    structured_queries = []
    for i, (english, general, localized) in enumerate(questions):
        structured_queries.append({
            'q_id': i + 1,
            'english': english,
            'sinhala_general': general,
            'sinhala_localized': localized
        })

    return structured_queries

In [18]:
def ask_gemini(prompt):
    response = model.generate_content(prompt)
    return response.text

In [19]:
sinhala_queries = generate_sinhala_llm_queries()

results = []  # store everything for your assignment

for q in sinhala_queries:
    q_id = q['q_id']

    english    = q['english']
    general    = q['sinhala_general']
    localized  = q['sinhala_localized']

    print(f"\n=== Question {q_id} ===")
    print("English:", english)

    # Gemini outputs
    gen_output  = ask_gemini(general)
    loc_output  = ask_gemini(localized)

    print("\nSinhala (General):", general)
    print("→ Response:")
    print(gen_output)

    print("\nSinhala (Localized):", localized)
    print("→ Response:")
    print(loc_output)

    print("-" * 40)

    # save to list
    results.append({
        "q_id": q_id,
        "english": english,
        "general_query": general,
        "general_response": gen_output,
        "localized_query": localized,
        "localized_response": loc_output
    })



=== Question 1 ===
English: What to serve my kid for breakfast?

Sinhala (General): මගේ දරුවාට උදේ ආහාරයට දෙන්නේ කුමක්ද?
→ Response:
දරුවෙකුට උදේ ආහාරය සඳහා දිය හැකි ආහාර කිහිපයක් පහත දැක්වේ. ඔබේ දරුවාගේ වයස, රුචි අරුචිකම් සහ සෞඛ්‍ය අවශ්‍යතා අනුව මේවා වෙනස් විය හැකියි.

*   ඕට්ස් (Oats): ඕට්ස් යනු දරුවන්ට ඉතාමත් ගුණදායක ආහාරයකි.
*   පළතුරු (Fruits): කෙසෙල්, ඇපල්, ස්ට්‍රෝබෙරි වැනි පළතුරු වර්ග විටමින් සහ තන්තු වලින් පොහොසත්ය.
*   එළවළු (Vegetables): කැරට්, පිපිඤ්ඤා වැනි එළවළු කැබලි උදෑසන ආහාරයට එකතු කරගත හැකිය.
*   බිත්තර (Eggs): බිත්තර ප්‍රෝටීන් වලින් පොහොසත් වන අතර දරුවාගේ වර්ධනයට උපකාරී වේ.
*   යෝගට් (Yogurt): යෝගට් වල ප්‍රෝබියොටික්ස් අඩංගු වන අතර එය දරුවාගේ ආහාර ජීර්ණයට හිතකර වේ.
*   පාන් (Bread): සම්පූර්ණ ධාන්‍ය පාන් (whole grain bread) කුඩා කැබලිවලට කපා දරුවාට දිය හැකිය.

මෙම ආහාර දරුවාට දීමට පෙර, දරුවාට අසාත්මිකතා (allergies) තිබේදැයි හොඳින් පරීක්ෂා කර බලන්න. එමෙන්ම දරුවාගේ වයසට සහ අවශ්‍යතාවයට ගැලපෙන ප්‍රමාණයෙන් ආහාර ලබාදීමට සැලකිලිමත් වන්න. තවද මේ සම්බන්ධයෙන් වෛද්‍ය උපදෙස් ලබාගැ